# Exercises due by EOD <span style="color:red">2018.12.13<span>

## goal

in this homework assignment we will work with `hadoop` and `spark` as well as `github` `pull request`s.

for several reasons:

+ because the final will be assigned on the 13th and we must push answers to you and grade them asap
+ `emr` is crazy expensive

this homework assignment will be a bit smaller and easier than others

## method of delivery

as mentioned in our first lecture, the method of delivery may change from assignment to assignment. we will include this section in every assignment to provide an overview of how we expect homework results to be submitted, and to provide background notes or explanations for "new" delivery concepts or methods.

this week you will be submitting the results of your homework via upload to your `s3` homework bucket and `commit`s on `github`

summary:

| exercise | deliverable | method of delivery |
|----------|-------------|--------------------|
| 1 | none | none |
| 2 | none | none |
| 3 | a file `super_hadooper.csv ` | uploaded to your `s3` homework submission bucket |
| 4 | a file `download_usaspending_sample.sh` | uploaded to your `s3` homework submission bucket |
| 5 | a file `load_usaspending_to_hdfs.sh` | uploaded to your `s3` homework submission bucket |
| 6 | none | none |

# exercise 1: set up a virtual development environment

the appendix of the `015_hadoop.ipynb` lecture includes several ways of creating a `hadoop` development environment. as much as possible, we will try to execute our `hadoop` commands in a local development environment (using `docker` or virtual machines) until we know they work; then we can pay the big bucks for an `emr` environment when needed.

if you run into issues, please check the lecture appendix first, and then reach out to us.


## 1.1: updating our `docker` vm memory size

these images require a log of memory, and the default amount of memory set aside for `docker` `containers` is 2GB

if you are using a windows or mac computer, increase your vm memory as discussed [here](https://stackoverflow.com/questions/44533319/how-to-assign-more-memory-to-docker-container). I recommend `8 GB` if your computer has more than `8GB` of memory, otherwise `4 GB`.

for linux, just increase memory by invoking `docker run` (when you do) with the command line flag `--memory=8g`


## 1.2: finding good local ports

let's create a `docker` `container` running the `cloudera` `hadoop` distribution (version 5.7 quickstart method). the main thing to figure out is what local ports we can map to the `container`'s internal ports. go to each of the following and see if any web app is running there:

+ port `8888`: http://localhost:8888
+ port `7180`: http://localhost:7180
+ port `8880`: http://localhost:8880

hopefully there are no applications running on those ports, and what we see at every url is

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=13Neuqun5a7GZQRpnWmZ5U6Yeh-5V3hGr"></div>

if you see an application (especially `jupyter` running on port `8888`, its default port) simply take the "problem" port and add 1 and try again (e.g. is anything running on http://localhost:8889? how about http://localhost:8890?). keep going until you have a series of unoccupied ports


## 1.3: running the `docker` `image`

take those unoccupied ports and fill them in below, then run the commands from the command line

```sh
HOST_PORT_HUE=9999
HOST_PORT_CLOUDERA_MANAGER=7180
HOST_PORT_TUTORIAL=8880

# if copy-paste doesn't work, try the one-line version below
docker run \
    --hostname=quickstart.cloudera \
    --privileged=true \
    --rm \
    -it \
    -p $HOST_PORT_HUE:8888 \
    -p $HOST_PORT_CLOUDERA_MANAGER:7180 \
    -p $HOST_PORT_TUTORIAL:80 \
    cloudera/quickstart:latest \
    /usr/bin/docker-quickstart
```

if the copy-paste didn't work, here's one that's all on one line (but less readable!)

```sh
# one-line version
docker run --hostname=quickstart.cloudera --privileged=true --rm -it -p $HOST_PORT_HUE:8888 -p $HOST_PORT_CLOUDERA_MANAGER:7180 -p $HOST_PORT_TUTORIAL:80 cloudera/quickstart:latest /usr/bin/docker-quickstart
```

this should download the `cloudera` `quickstart` `docker` `image` and then kick off a long stream of startup scripts. you will know you are done when the terminal looks like this:

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1TzvLGbGkvJlID19stE8NhI17aaLTP3a9"></div>

you can execute `hadoop` commands inside this `container` now:

```sh
hadoop fs -ls /
```

you can also verify that it all worked by going to `http://localhost:8888`, where `8888` is replaced with whatever port you found up above. if there is a web applciation `hue` running there:

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1zbnV1GmCBHx6XLGur17HLS3MVg4w6ReI"></div>

the username and password for this application are both `cloudera`

you're good to go!

when you're done with the `container` (after this assignment) exit the terminal with `exit`

##### there is nothing to submit here

# exercise 1: set up a virtual development environment <span style="color:red;font-weight:bold">ANSWERS</span>

follow the instructions above and in the lecture materials

<div style="border: 1px solid lightgrey;">

# exercise 2: read the rest of the `hadoop` lecture

read the rest of the `015_hadoop.ipynb` lecture. the assignments below will leverage some of the material presented there -- especially `hadoop fs` commands and `pyspark`.

the commands in the lecture are specifically designed to work in the `emr` environment -- which is slightly different than the `cloudera` environment. if you run into issues running commands, please reach out to me directly

##### there is nothing to submit for this question

# exercise 2: read the rest of the `hadoop` lecture <span style="color:red;font-weight:bold">ANSWERS</span>

no answers, just read the lecture

<div style="border: 1px solid lightgrey;">

# exercise 3: re-learning `bash` for `hdfs`

## 3.1: setup

do the following in your *cloudera development environment*!

let's create some giberish files on our local file system for use in interacting with the `hdfs`. simply execute the following code to create a number of local files we can reference in our commands.

*note: you may need to install `wget` with `yum install wget`*

```bash
mkdir /tmp/cachenet
cd /tmp/cachenet
wget -x www.google.com
wget -x www.nytimes.com
wget -x www.twitter.com/i/moments
wget -x www.twitter.com/i/notifications
wget -x www.facebook.com
wget -x www.youtube.com
wget -x www.youtube.com/feed/trending
wget -x www.espn.com
wget -x en.wikipedia.org
wget -x en.wikipedia.org/wiki/L33T
wget -x www.reddit.com
wget -x www.reddit.com/r/all
wget -x www.reddit.com/r/datascience
wget -x www.reddit.com/r/python
cd ~
```


## 3.2: commands

in the left column of the table below I have described some pretty common filesystem operations, and listed in the second column the `bash` commands to execute them. fill in the comparable commands (i.e. those you enter at a bash prompts) to perform the same general tasks on the `hdfs`. save the resulting three-column table as a `csv` with name `super_hadooper.csv`


### 3.2.1: quick note: running as an `hdfs` super user

some commands will require you to be running as the `hdfs` user instead of the `root` user -- this is a file permission control built in to `hadoop`. some distributions have the main `hadoop` user named `hadoop`; others named `hdfs`. `cloudera` has chosen `hdfs`.

to execute commands as the `hdfs` user, it is necessary to prefix your commands with `sudo -u hdfs`, which will execute the command as the `hdfs` linux user account (which is the `hdfs` super user). given a hadoop command you know, you can execute that same command by prefacing it like:

```
sudo -u hdfs hadoop [HADOOP CMD HERE]
```

e.g. `hadoop fs -ls /` would turn into `sudo -u hdfs hadoop fs -ls /`


### 3.2.2: read the manual!

feel free to check out the `hadoop fs` documentation. make sure you get the right version! check your `hadoop` version with `hadoop -version`. some of the versions we have access to*:

+ [the `cloudera` `docker` container: 2.6.0](http://hadoop.apache.org/docs/r2.6.0/hadoop-project-dist/hadoop-common/FileSystemShell.html)
+ [the `aws` `emr` instance: 2.8.5](http://hadoop.apache.org/docs/r2.8.5/hadoop-project-dist/hadoop-common/FileSystemShell.html)


### 3.2.3: the command table

| task description | bash command | `hadoop` command |
|-|-|-|
| ask for help re: the `ls` command | ls --help | |
| make a directory named `/tmp` | `mkdir -p /tmp` | |
| create an empty file `/tmp/test` | `touch /tmp/test` | |
| list the contents of `/tmp` | `ls /tmp` | |
| add group write permission for `/tmp/test` | `chmod g+w /tmp/test` | |
| change the owner of `/tmp/test` to `hadoop` | `chown hadoop /tmp/test` | |
| remove a file `/tmp/test` which you don't own | `sudo rm /tmp/test` | |
| copy local `/tmp/cachenet` to `hdfs` | no meaningful analog | |
| print the final lines of `/tmp/cachenet/www.nytimes.com/index.html` | `tail /tmp/cachenet/www.nytimes.com/index.html` | |
| echo `hello world` to `/tmp/hello_world.txt` | `echo hello world > /tmp/hello_world.txt` | |
| print `/tmp/hello_world.txt` to `stdout` | `cat /tmp/hello_world.txt` | |
| get the free and used space | `df -h` | |
| count the files and dirs under `/tmp` | no equivalent! |
| remove `/tmp/hello_world.txt` | `rm /tmp/hello_world.txt` | |
| recursively remove directory `/tmp/cachenet` | `rm -r /tmp/cachenet` | |


##### upload `super_hadooper.csv` to your `s3` bucket

# exercise 3: re-learning `bash` for `hdfs` <span style="color:red;font-weight:bold">ANSWERS</span>

| task description | bash command | `hadoop` command |
|-|-|-|
| ask for help re: the `ls` command | ls --help | `hadoop fs -help ls` |
| make a directory named `/tmp` | `mkdir -p /tmp` | `hadoop fs -mkdir -p /tmp` |
| create an empty file `/tmp/test` | `touch /tmp/test` | `hadoop fs -touchz /tmp/test` |
| list the contents of `/tmp` | `ls /tmp` | `hadoop fs -ls /tmp` |
| add group write permission for `/tmp/test` | `chmod g+w /tmp/test` | `hadoop fs -chmod g+w /tmp/test` |
| change the owner of `/tmp/test` to `hadoop` | `chown hadoop /tmp/test` | `hadoop fs -chown hadoop /tmp/test` |
| remove a file `/tmp/test` which you don't own | `sudo rm /tmp/test` | `hadoop fs -rm /tmp/test` |
| copy local `/tmp/cachenet` to `hdfs` | no meaningful analog | `hadoop fs -put /tmp/cachenet /tmp` |
| print the final lines of `/tmp/cachenet/www.nytimes.com/index.html` | `tail /tmp/cachenet/www.nytimes.com/index.html` | `hadoop fs -tail /tmp/cachenet/www.nytimes.com/index.html` |
| echo `hello world` to `/tmp/hello_world.txt` | `echo hello world > /tmp/hello_world.txt` | `echo hello world `&#124;` hadoop fs -put - /tmp/hello_world.txt` |
| print `/tmp/hello_world.txt` to `stdout` | `cat /tmp/hello_world.txt` | `hadoop fs -cat /tmp/hello_world.txt` |
| get the free and used space | `df -h` | `hadoop fs -df -h` |
| count the files and dirs under `/tmp` | no easy analog | `hadoop fs -count /tmp/` |
| remove `/tmp/hello_world.txt` | `rm /tmp/hello_world.txt` | `hadoop fs -rm /tmp/hello_world.txt` |
| recursively remove directory `/tmp/cachenet` | `rm -r /tmp/cachenet` | `hadoop fs -rm -r /tmp/cachenet` |

<div style="border: 1px solid lightgrey;">

# exercise 4: downloading a sample `usaspending` dataset

## 4.1: background

[USA Spending](https://www.usaspending.gov/#/about) is an excellent open data source which offers details on government expenditures (e.g. IT accessory purchases, contract work). the datasets are freely available for download, either as one-off files (see https://www.usaspending.gov/#/download_center/award_data_archive), or as a `postgres` database (see [this page](https://files.usaspending.gov/database_download/)).

let's get a file to work with. we want one that is small enough we can still work with it in development environments like our `docker` containers or our `vm`s. the entire dataset is around 90GB, so something we would probably want to work with in a database or distributed environment like `hadoop`


## 4.2: downloading

on [the award data archive page](https://www.usaspending.gov/#/download_center/award_data_archive), make sure the "Fiscal Year" dropdown is set to 2019. the top line in the table of links will be something like `2019_all_Contracts_Full_YYYYMMDD.zip`, where (as of writing) the `YYYYMMDD` "as of" date is `20181114` -- that date will update as the underlying datasets are modified and updated.

as of writing, the 2019 contracts zip file `url` was: https://files.usaspending.gov/award_data_archive/2019_all_Contracts_Full_20181114.zip

1. write a simple `curl` or `wget` command to download this to a linux instance
1. write a simple `unzip` statement to unzip that file into a `csv`


## 4.3: record format

there is a complete data dictionary for this record online [here](https://www.usaspending.gov/#/download_center/data_dictionary), but the most important fields for our interest are:

+ unique record identifiers
    + `parent_award_id`: "The identifier of the procurement award under which the specific award is issued (such as a Federal Supply Schedule). Term currently applies to procurement actions only"
    + `award_id_piid`: "The unique identifier of the specific award being reported."
    + `modification_number`: "The identifier of an action being reported that indicates the specific subsequent change to the initial award."
    + `transaction_number`: "Tie Breaker for legal, unique transactions that would otherwise have the same key."
+ other useful fields
    + `action_date`: "The date the action being reported was issued / signed by the Government or a binding agreement was reached."
    + `last_modified_date`: "The last modified date captures the change date."
    + `current_total_value_of_award`: "Total amount obligated to date on a contract, including the base and exercised options."
    + `awarding_agency`: "The name associated with a department or establishment of the Government as used in the Treasury Account Fund Symbol (TAFS)."
    + `recipient_name`: "The name of the awardee or recipient that relates to the unique identifier. For U.S. based companies, this name is what the business ordinarily files in formation documents with individual states (when required)."


## 4.4: a simple in-memory filter

this dataset includes multiple snapshots throught time of expenditures -- as contracts get updated, we get new *modifications* and we get new `modification_number` values. for example, given the downloaded `csv` as of part 2 above:

```python
import pandas as pd

df = pd.read_csv('2019_all_Contracts_Full_20181115_1.csv')

# a special award with multiple modifications
multi_mods = df[
    (df.parent_award_id == 'GS07F0284N')
    & (df.award_id_piid == 'DJJ15FUSA830157')
]
multi_mods[[
    'parent_award_id', 'award_id_piid', 'modification_number',
    'transaction_number', 'current_total_value_of_award', 'action_date',
    'last_modified_date', 'awarding_agency_name', 'recipient_name', 
]].sort_values(by='last_modified_date')
```

will result in the following table:

| index | parent_award_id | award_id_piid | modification_number | transaction_number | current_total_value_of_award | action_date | last_modified_date | awarding_agency_name | recipient_name |
|-|-|-|-|-|-|-|-|-|-|
9914 | GS07F0284N | DJJ15FUSA830157 | P00001 | 0.0 | 174724.59 | 2018-11-05 | 2018-11-05 15:59:15 | DEPARTMENT OF JUSTICE (DOJ) | ALL U NEED TEMPORARY SERVICES |
10811 | GS07F0284N | DJJ15FUSA830157 | P00002 | 0.0 | 174724.59 | 2018-11-05 | 2018-11-05 16:06:42 | DEPARTMENT OF JUSTICE (DOJ) | ALL U NEED TEMPORARY SERVICES |
9681 | GS07F0284N | DJJ15FUSA830157 | P00003 | 0.0 | 174724.59 | 2018-11-05 | 2018-11-05 16:13:37 | DEPARTMENT OF JUSTICE (DOJ) | ALL U NEED TEMPORARY SERVICES |
9843 | GS07F0284N | DJJ15FUSA830157 | P00004 | 0.0 | 174724.59 | 2018-11-05 | 2018-11-05 16:21:04 | DEPARTMENT OF JUSTICE (DOJ) | ALL U NEED TEMPORARY SERVICES |
9712 | GS07F0284N | DJJ15FUSA830157 | P00005 | 0.0 | 174724.59 | 2018-11-05 | 2018-11-05 16:26:26 | DEPARTMENT OF JUSTICE (DOJ) | ALL U NEED TEMPORARY SERVICES |
9883 | GS07F0284N | DJJ15FUSA830157 | P00006 | 0.0 | 174724.59 | 2018-11-05 | 2018-11-05 16:30:53 | DEPARTMENT OF JUSTICE (DOJ) | ALL U NEED TEMPORARY SERVICES |

one thing we might care to do is filter down this larger dataset of all the updated versions of records into only the most recent value (as defined by `modifcation_number` and `transaction_number`, or `last_modified_date`).

we can do this with the following `python` code (assuming the dataframe has been loaded with `read_csv` as above):

```python
most_recent = df \
    .fillna({'parent_award_id': 'paid_nan', 'award_id_piid': 'aip_nan'}) \
    .sort_values(by=['parent_award_id', 'award_id_piid', 'last_modified_date']) \
    .groupby(['parent_award_id', 'award_id_piid']) \
    .last()
```

for the file downloaded via the link https://files.usaspending.gov/award_data_archive/2019_all_Contracts_Full_20181114.zip, we had

+ 78,497 records in our *full*, unmodified `df` created with the `read_csv` function as above
+ 70,702 most-recent records in our `most_recent` dataframe created as above


## 4.5: what to submit

this one's simple: take the `curl` or `wget` statement (line 1) and the `unzip` statement (line 2) that you used in part 2 and write those two lines to a file named `download_usaspending_sample.sh`


##### upload `download_usaspending_sample.sh` to your `s3` homework submission bucket

# exercise 4: downloading a sample `usaspending` dataset <span style="color:red;font-weight:bold">ANSWERS</span>

the `sh` script is

```sh
wget https://files.usaspending.gov/award_data_archive/2019_all_Contracts_Full_20181114.zip
unzip 2019_all_Contracts_Full_20181114.zip
```

<div style="border: 1px solid lightgrey;">

# exercise 5: loading `usaspending` dataset into `hdfs`

**inside your development environment you created above**, use the commands from the previous problem to download the 2019 contracts `zip` file and `unzip` to get a `csv` inside that development environment (e.g. within the container, vm, or `aws` `emr` instance). in that environment you may need to `yum` or `apt` install `wget`, `curl`, or `unzip`.

let's load that `csv` file into `hdfs`. with one `hadoop` command, create a directory `/data` you can use to store data. with a second, `put` your `csv` into `hdfs` in your development environment at path `/data/usaspending.csv`

write the statements you used to create the `/data` directory and `put` that file into `hdfs` to a file named `load_usaspending_to_hdfs.sh`

##### upload `load_usaspending_to_hdfs.sh` to your `s3` submission bucket

# exercise 5: loading `usaspending` dataset into `hdfs` <span style="color:red;font-weight:bold">ANSWERS</span>

```sh
hadoop fs -mkdir /data
hadoop fs -put 2019_all_Contracts_Full_20181115_1.csv /data/usaspending.csv
```

<div style="border: 1px solid lightgrey;">

# exercise 6: `pyspark` filter job for `usaspending` dataset

let's duplicate our `pandas` filter work above with `pyspark`. we will jump through some hoops to use the `pyspark` `sql` `api` -- a time-tested way to groupby, aggregate, and window (what we need to do!)

to do this we will use some libraries that are available in `spark` version 2+, so unfortunately our cheap development environments won't do. we'll spin up an `emr` instance instead! this is just a walkthrough, so you will only be executing commands -- this is to keep the cost of running your cluster low. **don't forget to terminate your cluster when you're done!!**


## 6.1: launch an `emr` cluster

spin up an `aws` `emr` cluster with `spark` installed. on the "Create Cluster - Quick Options" page, set the following

+ whatever name you want
+ select the `spark` software configuration option
+ change the instance type to `m4.large`
+ change the number of nodes from 3 (master and 2 core) down to 1 (just one master)
+ pick an `ssh` key pair

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1zk8-keZGm-H_vHTGqtY34cK_BJXAdXJu"></div>


## 6.2: getting the data (again)

run the steps from the previous exercises to get a file in `hdfs` named `/data/usaspending.csv`


## 6.3: launching `pyspark`

make an `ssh` connection and launch `pyspark` from the command line. you will receive errors about logging; ignore them for now.


## 6.4: loading a `csv`

in `spark` v2 and beyond, you can load `csv`s from `hdfs` easily with the `spark.read.csv` function. do that!

```python
df = spark.read.csv('/data/usaspending.csv', header=True)
```

you can also count the records to make sure you have the same number as you saw in your local (`pandas`) investigations:

```python
# at time of writing, this returned 78,497
df.count()
```


## 6.5: create a `Window` object

the way that we will do our group by and sort is with a `Window` object (like the window operations in traditional `sql`. we do this in three steps:

first, create a `spark` `Window` object:

```python
from pyspark.sql.functions import col
from pyspark.sql.window import Window

w = Window()
```

second, we will update that `Window` to partition all of our records by the keys we care about ("partition" is synonymous with "group by" here):

```python
from pyspark.sql.functions import col
from pyspark.sql.window import Window

w = (Window()
  .partitionBy('parent_award_id', 'award_id_piid'))
```

finally, our `Window` object which partitions records by our favored keys shoudl be *ordered*, so that we know our individual groups are sorted. we'll use the updated `Window` object's `.orderBy` method, and we will order based on the `col`umn `last_modified_date`, descending (so largest / most recent value on top)

```python
from pyspark.sql.functions import col
from pyspark.sql.window import Window

w = (Window()
  .partitionBy('parent_award_id', 'award_id_piid')
  .orderBy(col('last_modified_date').desc())
```


## 6.6: using that `Window` object to calculate order within groups

the `Window` object we just created is something that can be used to calculate aggregation functions within the defined windows. for example, we could calculate the `sum` or `average` of different values within the `groups` defined by the above `partitionBy` partitions.

in this instance, we're looking to find the first record within each group (we have descending error), and we can do this by calculating the `row_number`. we will add this as a new column called "rn" to a dataframe, and save the result to a new dataframe of morst recent records

```python
from pyspark.sql.functions import col, row_number

mr = (df
  .withColumn("rn", row_number().over(w)))
```

after adding that column, we will filter the overall dataset down to records where the `rn` value is 1 (the first row in the window group

```python
from pyspark.sql.functions import col, row_number

mr = (df
  .withColumn("rn", row_number().over(w))
  .where(col("rn") == 1))
```

finally, from this filtered dataset we will select all columns

```python
from pyspark.sql.functions import col, row_number

mr = (df
  .withColumn("rn", row_number().over(w))
  .where(col("rn") == 1)
  .select("*"))
```

you can also count the records to make sure you have the same number as you saw in your local (`pandas`) investigations:

```python
# at time of writing, this returned 70,702
mr.count()
```


## 6.7: save the results

now that we've filtered our dataset, we can immediately save the results for other downstream computation. each dataframe has a `.write.csv` method associated with it, and the first argument is the directory in `hdfs` in which we will save the output. *note: the output is not a single `csv`, but rather a directory containing a collection of files, each being the output of an chunk of records*

```python
out_dir = '/data/usaspending_most_recent/'
mr.write.csv(out_dir)
```


## 6.8: the whole thing, together

```python
from pyspark.sql.functions import row_number, col
from pyspark.sql.window import Window

df = spark.read.csv('/data/usaspending.csv', header=True)
df.count()

w = (Window()
  .partitionBy("parent_award_id", "award_id_piid")
  .orderBy(col("last_modified_date").desc()))

mr = (df
  .withColumn("rn", row_number().over(w))
  .where(col("rn") == 1)
  .select("*"))
mr.count()

out_dir = '/data/usaspending_most_recent/'
mr.write.csv(out_dir)
```


## 6.9: check the output

exit the `pyspark` session with

```python
exit()
```

back on the `emr` cluster command line, check the contents of the `mr.write.csv` command:

```sh
hadoop fs -ls /data/usaspending_most_recent
```


## 6.10: <span style="color:red;font-weight:bold">SHUT OFF YOUR CLUSTER</span>

don't forget to `terminate` your `emr` cluster!!


##### there is nothing to submit here

# exercise 6: `pyspark` filter job for `usaspending` dataset <span style="color:red;font-weight:bold">ANSWERS</span>

it's just a walkthrough, so follow the instructions! the code here is

```python
from pyspark.sql.functions import row_number, col
from pyspark.sql.window import Window

df = spark.read.csv('/data/usaspending.csv', header=True)
df.count()

w = (Window()
  .partitionBy("parent_award_id", "award_id_piid")
  .orderBy(col("last_modified_date").desc()))

mr = (df
  .withColumn("rn", row_number().over(w))
  .where(col("rn") == 1)
  .select("*"))
mr.count()

out_dir = '/data/usaspending_most_recent/'
mr.write.csv(out_dir)
```

<div style="border: 1px solid lightgrey;">

# exercise 7: resolving an issue with a `commit` to a `branch`

recall from a previous assignment that `github` -- *not* `git` itself -- allows you to create issues to track bugs and feature requests, and to close issues with commits. previously we closed issues by pushing changes to `master`; now we will resolve them by pushing changes to a `branch` and using the `github` web interface to close them via a `pull request`.

I have already added an issue to your repositories requesting a simple change be made. the issue's title is **die, 511 homeworks!**, and the goal is to create a new file called `status_report` which logs that we've finished our homework for MATH 511.


## 7.1: viewing and assigning the issue

log in to `github` and click on the "issues" tab, and open the issue I created for you. in particular, I want you to **assign** it to yourself -- click on the "assign yourself" link on the issues page.

also, **make note of the issue number!**. you will need to include that number in your `commit` `message`


## 7.2: creating a `branch`

in your local repo, create a new `branch` named `statusreport` and check that `branch` out


## 7.3: adding a status report

create a new file named `status_report.txt` with only one line: `homework is finished`.

`add` that file to tracking and `commit` it to your `statusreport` `branch`. for your `commit` message, use

```
# replace #N with your issue number!!
status report: initial commit, fixes #N
```

with `#N` above replaced by the issue number you have in `github`.


`push` this `commit` to `github`. remember: you don't push `branch`es to `origin master`, you push them to `origin [branch name]`!)


## 7.4: check out the new `pull request` on `github`

log in to `github`. `github` noticed that the `commit` you just pushed has a reference to an issue (`"fixes #5"`) and provides you with a way to view the newly-created `pull request`

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=14RoWxzI2q5nUwRquQdrTxw9nyUiI7LCt" width="700px"></div>

click on the "compare & pull request" button to go to the "Open a pull request" page that `github` has generated for us:

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1WQRvvuQlAn5bnYKcFne_Rdu1hO-x3kcn" width="700px"></div>

this page is a form that will allow us to create a "pull request" -- a `github` concept that is, basically, a web-based merge of `branch`es resolving `issue`s. click the "create pull request" button to create an official pull request.

the page we are looking at will then update to a pull request. `github` will first calculate whether or not the changes in this `branch` can be directly merged into the `master` branch. if so, it will give us the ability to do the `merge` through the web console rather than the command line (pretty cool!)

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1BEUglXC0bjFLlCvziqNYTwNSapjmiq1f" width="700px"></div>


## 7.5: merge!

click that "Merge pull request" button and let 'er rip! the result is a successful pull request, and the webpage will update inplace:

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1KKGex6RXsED75XWsNrO5q6RXydnjVs-9" width="700px"></div>


## 7.6: view the new `merge` `commit` on the command line

the changes we just made to `master` in that pull request were all done on `github`. our `remote` knwos about them, but our `local` doesn't yet!

back in your local repo, `checkout` `master` and `pull` the `remote` changes. you should see the merge commit we just created completely online on `github` merging your `statusreport` `branch` with `master`, like this:

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1GDsYUK8vQg-c_UE9MAgbPj7C-y2_jA2s" width="700px"></div>


##### submission will be verified via `github`

# exercise 7: resolving an issue with a `commit` to a `branch` <span style="color:red;font-weight:bold">ANSWERS</span>

locally,

```sh
# the below can be combined into one command:
#    git checkout -B statusreport
git branch statusreport
git checkout statusreport

echo "homework is finished" > statusreport.txt

git add statusreport.txt
git commit -m 'status report: initial commit, fixes #N'
git push origin statusreport
```

then, go through the items listed above on `github`. once the `pull request` has been `merge`d into `master`, back in your local repo do

```sh
git checkout master
git pull
git gr
```

<div style="border: 1px solid lightgrey;">

# exercise 8: fill out a year-end course survey

I'd like your feedback on the course -- please fill out the form at https://goo.gl/forms/3Psw2IRUYs1aoO8c2. this is 100% anonymous and not mandatory

# exercise 8: fill out a year-end course survey <span style="color:red;font-weight:bold">ANSWERS</span>

no answers here

<div style="border: 1px solid lightgrey;">